# La colonne "brand"

In [2]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

import re
import string

import spacy
from spacy.tokens import Token
from spacy.attrs import ORTH
from unidecode import unidecode



C:\Users\hp\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


## Scrapper le site Chrono24

In [3]:

# URL de la page à scraper
url = 'https://www.chrono24.com/search/browse.htm'

# Définir des en-têtes HTTP pour la requête
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
}

# Envoyer une requête GET pour récupérer le contenu de la page
response = requests.get(url, headers=headers)

# Vérifier si la requête a été réussie
if response.status_code == 200:
    # Analyser le contenu HTML de la page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Dictionnaire pour stocker les marques par lettre
    brands_by_letter = {}

    # Parcourir chaque section
    sections = soup.find_all('section', {'class': 'm-b-7'})

    for section in sections:
        # Trouver la lettre associée à cette section
        letter_header = section.find('h2', {'class': 'bg-surface-highlight'})
        if letter_header:
            letter = letter_header.get_text().strip()
            if letter not in brands_by_letter:
                brands_by_letter[letter] = []
            
            # Trouver toutes les balises <li> dans cette section et ajouter les marques à la liste correspondante
            for li_tag in section.find_all('li', {'class': 'col-xs-12 col-sm-8 col-md-6'}):
                brand_name = li_tag.get_text().strip()
                brands_by_letter[letter].append(brand_name)

    # Créer un DataFrame à partir du dictionnaire
    df_list = []
    for letter, brands in brands_by_letter.items():
        for brand in brands:
            df_list.append({'Letter': letter, 'Brands': brand})

    df = pd.DataFrame(df_list)

    # Afficher le DataFrame
    print(df)

    # Enregistrer le DataFrame dans un fichier CSV local
    csv_file_path = 'brands_by_letter.csv'
    df.to_csv(csv_file_path, index=False, encoding='utf-8')

    print(f"Les marques ont été enregistrées dans le fichier {csv_file_path}")
else:
    print(f"Échec de la requête. Code de statut: {response.status_code}")


    Letter            Brands
0        A  A. Lange & Söhne
1        A         ABP Paris
2        A          Accutron
3        A  AD-Chronographen
4        A         Aerowatch
..     ...               ...
543      A  Zeno-Watch Basel
544      A            ZentRa
545      A          Zeppelin
546      A            Zodiac
547      A               ZRC

[548 rows x 2 columns]
Les marques ont été enregistrées dans le fichier brands_by_letter.csv


In [4]:

# Fonction pour extraire la première lettre d'une chaîne
def extract_first_letter(brands):
    return brands[0]

# Appliquer la fonction à la colonne 'Brand' pour extraire la première lettre
df['First_Letter'] = df['Brands'].apply(extract_first_letter)
df = df[['Letter', 'Brands', 'First_Letter']]

# Afficher le DataFrame avec les lettres associées
print(df)


    Letter            Brands First_Letter
0        A  A. Lange & Söhne            A
1        A         ABP Paris            A
2        A          Accutron            A
3        A  AD-Chronographen            A
4        A         Aerowatch            A
..     ...               ...          ...
543      A  Zeno-Watch Basel            Z
544      A            ZentRa            Z
545      A          Zeppelin            Z
546      A            Zodiac            Z
547      A               ZRC            Z

[548 rows x 3 columns]


In [154]:
df.head(50)

,Letter,Brands,First_Letter
0,A,A. Lange & Söhne,A
1,A,ABP Paris,A
2,A,Accutron,A
3,A,AD-Chronographen,A
4,A,Aerowatch,A
5,A,Aigle,A
6,A,Aigner,A
7,A,Alain Silberstein,A
8,A,Alexander Shorokhoff,A
9,A,Alfred Dunhill,A


In [155]:
#df.to_csv('df_new.csv', index=False, encoding='latin1')

In [5]:
# Créer un dictionnaire pour mapper chaque lettre à un ensemble de marques
letter_to_brands = {}

# Parcourir le DataFrame pour associer chaque lettre à un ensemble de marques
for index, row in df.iterrows():
    letter = row['First_Letter']  
    brand = row['Brands']
    if letter not in letter_to_brands:
        letter_to_brands[letter] = set()  # Initialiser un ensemble vide si la lettre n'existe pas encore
    letter_to_brands[letter].add(brand)  # Ajouter la marque à l'ensemble correspondant

# Afficher les marques associées à chaque lettre
for letter, brands in letter_to_brands.items():
    print(f"Lettre {letter}: {', '.join(brands)}")

Lettre A: ABP Paris, Armin Strom, Audemars Piguet, Azzaro, Alexander Shorokhoff, Alfred Dunhill, Aristo, Armand Nicolet, Atlantic, Alain Silberstein, Artya, Azimuth, Alfred Rochat & Fils, Angular Momentum, Andreas Strehler, AD-Chronographen, Askania, A. Lange & Söhne, Alpina, Armani, Avier, Aigle, Accutron, Arnold & Son, Ateliers deMonaco, Apple, Artisanal, Anonimo, Aigner, Angelus, Aquastar, Andersen Genève, Auricoste, Auguste Reymond, Aquanautic, Aerowatch
Lettre B: Berne, Baume & Mercier, Behrens, Breil, Bomberg, Bifora, Bulova, Ball, Borgeaud, Baltic, Bruno Söhnle, Boucheron, Benzinger, Boegli, Blacksand, Bell & Ross, Breitling, Burberry, Bedat & Co, Bogner Time, Brior, Blancpain, Black-Out Concept, B.R.M, Barington, Bulgari, Bunz, Bovet, Bertolucci, Balmain, Breguet, BWC-Swiss, Beuchat, Benrus, Blancier, Boldr, Bremont
Lettre b: blu
Lettre C: Cronus, Chopard, Casio, Christopher Ward, Cortébert, Concord, Century, Cyrus, Carl F. Bucherer, Cartier, Christophe Claret, Carlo Ferrara, C

In [112]:
os.getcwd()

'C:\\Users\\hp'

In [6]:
#path = r"C:\Users\arbif\Documents\M2 IMSD\big data\chrono24_reviews\chrono24_reviews"
#path1 = r"C:\Users\arbif\Documents\M2 IMSD\big data\archive"

reviews = pd.read_excel("chrono24_reviews.xlsx")
sellers = pd.read_excel("chrono24_sellers.xlsx")

In [7]:
reviews.head(50)

,brand,buyer_comment,buyer_country_code,review_date,review_url,reviews month,reviews year,has_comment,seller_uid,buyer_country
0,Oris Divers Sixty-Five,Everything great.,DE,13/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany
1,\n TAG Heuer Carrera Cal. 5 aut...,Super fast delivery. Goods are perfect and co...,DE,13/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany
2,Tissot T-Sport Quickster Chronograph,"Exactly as described, super fast delivery, I w...",DE,11/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,1,10003,Germany
3,Tissot PRS 516 Chronograph,-,DE,11/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,0,10003,Germany
4,Frederique Constant Vintage Rally Healey,-,FR,06/04/2020,https://www.chrono24.com/dealer/checkout-ratin...,April,2020.0,0,10003,France
5,Hamilton Jazzmaster GMT,-,ES,30/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,0,10003,Spain
6,Zenith PILOT Type 20 Chronograph Extra Special,Excellent support from this company. I had a p...,UK,27/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,1,10003,United Kingdom
7,\n Union Glashütte 1893 sub second,perfect deal,DE,26/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,1,10003,Germany
8,\n Alpina Startimer Pilot automa...,Quite fast delivery !!!!,CY,25/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,1,10003,Cyprus
9,"Oris Aquis Date 43,5 mm",Everything perfect! Gladly again. Still had ...,DE,19/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,2020.0,1,10003,Germany


In [8]:
reviews['brand'].head(50)

0                                Oris Divers Sixty-Five
1     \n               TAG Heuer Carrera Cal.  5 aut...
2                  Tissot T-Sport Quickster Chronograph
3                            Tissot PRS 516 Chronograph
4              Frederique Constant Vintage Rally Healey
5                               Hamilton Jazzmaster GMT
6        Zenith PILOT Type 20 Chronograph Extra Special
7      \n               Union Glashütte 1893 sub second
8     \n               Alpina Startimer Pilot automa...
9                               Oris Aquis Date 43,5 mm
10                                        Montblanc ...
11                                             Oris ...
12    \n               Day, this year ...\n         ...
13                                           Alpina ...
14                              Frederique Constant ...
15      \n               Alpina ...\n                  
16                                          Bomberg ...
17                                             M

In [9]:
#supprimer les espaces de début
reviews['brand'] = reviews['brand'].str.strip()

In [10]:
reviews['brand'].head(50)

0                             Oris Divers Sixty-Five
1           TAG Heuer Carrera Cal.  5 automatic 36mm
2               Tissot T-Sport Quickster Chronograph
3                         Tissot PRS 516 Chronograph
4           Frederique Constant Vintage Rally Healey
5                            Hamilton Jazzmaster GMT
6     Zenith PILOT Type 20 Chronograph Extra Special
7                    Union Glashütte 1893 sub second
8         Alpina Startimer Pilot automatic - limited
9                            Oris Aquis Date 43,5 mm
10                                     Montblanc ...
11                                          Oris ...
12                                Day, this year ...
13                                        Alpina ...
14                           Frederique Constant ...
15                                        Alpina ...
16                                       Bomberg ...
17                                          Mido ...
18                                        Bulo

In [11]:
# Initialiser la colonne 'marque' vide
reviews['marque'] = ''
df['Brands'] = df['Brands'].apply(lambda x: unidecode(x).lower())
reviews['brand'] = reviews['brand'].apply(lambda x: unidecode(x).lower())

# Boucle pour vérifier si chaque valeur de df['Brands'] est présente dans reviews['brand']
for i, review_brand in enumerate(reviews['brand']):
    for brand in df['Brands']:
        if brand.lower() in review_brand.lower():
            reviews.at[i, 'marque'] = brand
            break

# Afficher le résultat
print(reviews)

                                               brand  \
0                             oris divers sixty-five   
1           tag heuer carrera cal.  5 automatic 36mm   
2               tissot t-sport quickster chronograph   
3                         tissot prs 516 chronograph   
4           frederique constant vintage rally healey   
...                                              ...   
175963                    alpina vintage manual wind   
175964                              elysee automatic   
175965                        oris date, manual wind   
175966  kasper chronograph, manual wind, ref: 444165   
175967                                    zenith ...   

                                            buyer_comment buyer_country_code  \
0                                       Everything great.                 DE   
1       Super fast delivery.  Goods are perfect and co...                 DE   
2       Exactly as described, super fast delivery, I w...                 DE   
3      

In [12]:
reviews['brand'].head(50)

0                             oris divers sixty-five
1           tag heuer carrera cal.  5 automatic 36mm
2               tissot t-sport quickster chronograph
3                         tissot prs 516 chronograph
4           frederique constant vintage rally healey
5                            hamilton jazzmaster gmt
6     zenith pilot type 20 chronograph extra special
7                    union glashutte 1893 sub second
8         alpina startimer pilot automatic - limited
9                            oris aquis date 43,5 mm
10                                     montblanc ...
11                                          oris ...
12                                day, this year ...
13                                        alpina ...
14                           frederique constant ...
15                                        alpina ...
16                                       bomberg ...
17                                          mido ...
18                                        bulo

In [ ]:
#reviews.to_csv('reviews_new.csv', index=False, sep=';')

## Base Kaggel 

In [13]:
#importer la base Kaggel
WatchesBrands = pd.read_csv("WatchesBrands.csv", encoding='latin-1')

In [14]:
# Boucle pour vérifier si chaque valeur de WatchesBrands['Brands'] est présente dans reviews['brand']
WatchesBrands['Brands'] = WatchesBrands['Brands'].apply(lambda x: unidecode(x).lower())
for i, review_brand in enumerate(reviews['brand']):
    for brand in WatchesBrands['Brands']:
        if brand.lower() in review_brand.lower():
            reviews.at[i, 'marque'] = brand
            break

In [15]:
reviews['brand'].head(50)

0                             oris divers sixty-five
1           tag heuer carrera cal.  5 automatic 36mm
2               tissot t-sport quickster chronograph
3                         tissot prs 516 chronograph
4           frederique constant vintage rally healey
5                            hamilton jazzmaster gmt
6     zenith pilot type 20 chronograph extra special
7                    union glashutte 1893 sub second
8         alpina startimer pilot automatic - limited
9                            oris aquis date 43,5 mm
10                                     montblanc ...
11                                          oris ...
12                                day, this year ...
13                                        alpina ...
14                           frederique constant ...
15                                        alpina ...
16                                       bomberg ...
17                                          mido ...
18                                        bulo

In [18]:
# Liste des marques spécifiques et leurs correspondances
specific_brands = {
    'Jaeger Lecoultre': 'Jaeger Lecoultre',
    'Jaeger-LeCoultre': 'Jaeger-LeCoultre',
    'Hartig': 'Hartig',
    'Swarovski': 'Swarovski',
    'Allemano': 'Allemano',
    'Ferrari': 'Ferrari',
    'Vertex': 'Vertex',
    'Duward': 'Duward Aquastar',
    'Aquastar': 'Aquastar',
    'Schild': 'Unicorn A. Schild',
    'William L': 'William Watch',
    'HALCON': 'HALCON',
    'Cedric Johner': 'Cedric',
    'Revue Thommen': 'Thommen review',
    'Boccia': 'Boccia',
    'Lancyl': 'Lancyl',
    'Aqualung': 'Aqualung',
    'Consul': 'Consul',
    'Ermano': 'Ermano',
    'Vacheron Constantin': 'Vacheron Const',
    'Moeris': 'Moeris',
    'Sagara': 'Sagara',
    'Excelsior Park': 'Excelsior Park',
    'Voumard': 'Voumard',
    'Rodania': 'Rodania',
    'H. Stern': 'H. Stern',
    'Hentschel': 'Hentschel',
    'Buren': 'Buren',
    'Lemania': 'Lemenia',
    'Zannetti': 'Zannetti',
    'Wilhelm Tell': 'Tell',
    'Gay Freres': 'Gay Freres',
    'DKNY': 'DKNY',
    'Ruhla': 'Ruhla',
    'Silvana': 'Silvana',
    'Huguenin': 'Huguenin',
    'Javil': 'Chronograph Suisse Javil',
    'Heroic18': 'Heroic18',
    'buffalo': 'buffalo',
    'ANCON': 'ANCON'
}

# Boucle pour vérifier et mettre à jour les marques
for i, review_brand in enumerate(reviews['brand']):
    for marque, correspondance in specific_brands.items():
        if correspondance.lower() in review_brand.lower():
            reviews.at[i, 'marque'] = marque
            reviews.at[i, 'brand'] = review_brand.lower().replace(correspondance.lower(), '').strip()
            break

print(reviews)

                                               brand  \
0                             oris divers sixty-five   
1           tag heuer carrera cal.  5 automatic 36mm   
2               tissot t-sport quickster chronograph   
3                         tissot prs 516 chronograph   
4           frederique constant vintage rally healey   
...                                              ...   
175963                    alpina vintage manual wind   
175964                              elysee automatic   
175965                        oris date, manual wind   
175966  kasper chronograph, manual wind, ref: 444165   
175967                                    zenith ...   

                                            buyer_comment buyer_country_code  \
0                                       Everything great.                 DE   
1       Super fast delivery.  Goods are perfect and co...                 DE   
2       Exactly as described, super fast delivery, I w...                 DE   
3      

In [19]:
# Transformer toutes les valeurs de la colonne 'marque' en minuscules et gérer les caractères spéciaux
reviews['marque'] = reviews['marque'].apply(lambda x: unidecode(x).lower())

In [20]:
# Supprimer les lignes vides dans la colonne "marque"
reviews_clean = reviews.dropna(subset=['marque'])
reviews_clean = reviews_clean[reviews_clean['marque'] != ''] 
print(reviews_clean)

                                           brand  \
0                         oris divers sixty-five   
1       tag heuer carrera cal.  5 automatic 36mm   
2           tissot t-sport quickster chronograph   
3                     tissot prs 516 chronograph   
4       frederique constant vintage rally healey   
...                                          ...   
175962                            rolex datejust   
175963                alpina vintage manual wind   
175964                          elysee automatic   
175965                    oris date, manual wind   
175967                                zenith ...   

                                            buyer_comment buyer_country_code  \
0                                       Everything great.                 DE   
1       Super fast delivery.  Goods are perfect and co...                 DE   
2       Exactly as described, super fast delivery, I w...                 DE   
3                                                      

In [21]:
nombre_de_marques_differentes = reviews['marque'].nunique()
print(f"Nombre de marques différentes: {nombre_de_marques_differentes}")

Nombre de marques différentes: 460


In [22]:
 reviews.describe(include='O')

,brand,buyer_comment,buyer_country_code,review_date,review_url,reviews month,buyer_country,marque
count,175968,175959,175956,174150,175968,174150,173783,175968
unique,8173,88702,140,1367,7185,12,67,460
top,rolex ...,-,DE,17/03/2020,https://www.chrono24.com/dealer/checkout-ratin...,March,Germany,rolex
freq,26473,75855,33413,1036,30,19272,33413,27754


## Supprimer les valeurs manquantes 

In [23]:
# Vérifier les valeurs manquantes dans toutes les colonnes
valeurs_manquantes_par_colonne = reviews.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(valeurs_manquantes_par_colonne)


Nombre de valeurs manquantes par colonne :
brand                    0
buyer_comment            9
buyer_country_code      12
review_date           1818
review_url               0
reviews month         1818
reviews year          1818
has_comment              0
seller_uid               0
buyer_country         2185
marque                   0
dtype: int64


In [24]:
reviews = reviews.dropna()

In [25]:
reviews = reviews[reviews['marque'].str.strip() != '']

In [26]:
reviews = reviews[reviews['marque'].str.strip() != '']

In [27]:
# Vérifier les valeurs manquantes dans toutes les colonnes
valeurs_manquantes_par_colonne = reviews.isnull().sum()
print("Nombre de valeurs manquantes par colonne :")
print(valeurs_manquantes_par_colonne)

Nombre de valeurs manquantes par colonne :
brand                 0
buyer_comment         0
buyer_country_code    0
review_date           0
review_url            0
reviews month         0
reviews year          0
has_comment           0
seller_uid            0
buyer_country         0
marque                0
dtype: int64


In [3]:
#reviews.to_excel('reviews_sellers.xlsx', index=False)

In [178]:
reviews.shape

(156635, 11)